In [12]:
# import dependencies
import json
import pandas as pd
import numpy as np

# file_dir = 'C:\Users\Christopher\Desktop\Class_Folder\Movies-ETL' does not work because of back slash characters
# windows Explorer users must convert to forward slash path
file_dir = "C:/Users/Christopher/Desktop/Class_folder/Movies-ETL/"


In [13]:
# Load JSON as a list of dictionaries rather than a import like from a CSV
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)
    
len(wiki_movies_raw)

7311

In [24]:
for i in range(210,230):
    temp = wiki_movies_raw[i]['title']
    year = wiki_movies_raw[i]['year']
    print(i, year, temp)

#    wiki_movies_raw[i]['title']

    
wiki_movies_raw[222]
# is the list alphabetical?  Soultaker is only 222 that would be too soon to reach letter S in alphabet 
# could be alphabetical within the category of movie ? AHA its sorted by YEAR first then Alpha on Title 

# this is too easy now - remember how hard it was when JSON was like a black box? ? ? 
    

210 1990 Shipwrecked
211 1990 A Shock to the System
212 1990 Short Time
213 1990 A Show of Force
214 1990 The Shrimp on the Barbie
215 1990 Sibling Rivalry
216 1990 Side Out
217 1990 Slumber Party Massacre III
218 1990 Sorority House Massacre II
219 1990 Snake Eater II: The Drug Buster
220 1990 Solar Crisis
221 1990 Somebody Has to Shoot the Picture
222 1990 Soultaker
223 1990 Spaced Invaders
224 1990 The Spirit of '76
225 1990 Spontaneous Combustion
226 1990 Spymaker: The Secret Life of Ian Fleming
227 1990 Stanley & Iris
228 1990 State of Grace
229 1990 Stella White


{'url': 'https://en.wikipedia.org/wiki/Soultaker_(film)',
 'year': 1990,
 'imdb_link': 'https://www.imdb.com/title/tt0100665/',
 'title': 'Soultaker',
 'Directed by': 'Michael Rissi',
 'Produced by': ['Connie Kingrey', 'Eric Parkinson'],
 'Screenplay by': 'Vivian Schilling',
 'Story by': ['Eric Parkinson', 'Vivian Schilling'],
 'Starring': ['Joe Estevez',
  'Vivian Schilling',
  'Gregg Thomsen',
  'Chuck Williams',
  "Robert Z'Dar",
  'David "Shark" Fralick'],
 'Music by': 'Jon McCallum',
 'Cinematography': 'James Rosenthal',
 'Edited by': ['Jason Coleman', 'Michael Rissi'],
 'Productioncompanies ': ['Pacific West Entertainment Group',
  'Victory Pictures'],
 'Distributed by': 'Action International Pictures',
 'Release date': ['October 26, 1990', '(', '1990-10-26', ')'],
 'Running time': '94 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$250,000',
 'Box office': '$43,982'}

In [33]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

ratings.head()

# these files take a long time to read in so try not to do this step too many times after it completes

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [41]:
kaggle_metadata.sample(n=10)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
7971,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 53, ...",NaN,41925,tt0099292,en,The Comfort of Strangers,An Italian diplomat's son follows and seduces ...,...,1990-01-01,1244381.0,107.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A story of passionate obsession.,The Comfort of Strangers,False,5.9,24.0
15281,False,"{'id': 50457, 'name': 'Sex and the City Collec...",100000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.sexandthecitymovie.com/,37786,tt1261945,en,Sex and the City 2,"Carrie, Charlotte, and Miranda are all married...",...,2010-05-26,288347692.0,146.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Carrie on.,Sex and the City 2,False,5.5,434.0
41411,False,NaN,0,"[{'id': 10402, 'name': 'Music'}, {'id': 99, 'n...",http://deeppurple.com,63244,tt0452009,en,Classic Albums: Deep Purple - Machine Head,This musical biography tells the story of the ...,...,2002-12-02,0.0,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Classic Albums,Classic Albums: Deep Purple - Machine Head,False,4.8,3.0
29332,False,NaN,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",NaN,229182,tt2543336,en,The Harvest,Maryann moves in with her grandparents after s...,...,2013-10-19,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"First the Fall, then The Harvest.",The Harvest,False,5.9,69.0
30036,False,NaN,0,"[{'id': 16, 'name': 'Animation'}, {'id': 18, '...",NaN,118794,tt0097118,ru,Корова,Animated short based on the story of the same ...,...,1990-12-31,0.0,10.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,The Cow,False,6.8,6.0
20703,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,158739,tt2027064,en,Blue Caprice,A narrative feature film inspired by the event...,...,2013-09-12,0.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some killers are born. Others are driven to it.,Blue Caprice,False,5.8,42.0
13128,False,NaN,130000000,"[{'id': 18, 'name': 'Drama'}]",NaN,6972,tt0455824,en,Australia,"Set in northern Australia before World War II,...",...,2008-11-18,49554002.0,165.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Welcome to Australia!,Australia,False,6.3,712.0
6075,False,NaN,70000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,9567,tt0314353,en,Tears of the Sun,Navy SEAL Lieutenant A.K. Waters and his elite...,...,2003-03-07,85632458.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He was trained to follow orders. He became a h...,Tears of the Sun,False,6.4,582.0
31867,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,230588,tt2711672,en,Peace After Marriage,"Desperate for companionship, a lonely, young P...",...,2013-10-28,0.0,86.0,[],Released,Only in New York.,Peace After Marriage,False,3.5,2.0
4488,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,19118,tt0097731,en,Let It Ride,A gambler used to losing is having a very good...,...,1989-08-18,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He drinks. He smokes. He gambles. He curses. H...,Let It Ride,False,7.1,28.0
